# VexO (Veritas Ex Oculi)
> Truth from the eyes
***



<div class="alert alert-block alert-info">
<b>Tip:</b> Tes satu gambar yang persenannya paling salah, validasi dari situ.
</div>

## Inisialisasi Project

### Import dependencies
Import dependencies dari program, seperti OpenCV, Numpy, Pytesseract, Keras, dan Scikit-learn.

In [ ]:
import cv2
import numpy as np
import pytesseract
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress info and warnings, only errors will be shown
from keras.api.applications import Xception
from keras.api.models import Sequential, load_model
from keras.api.layers import Dense, Input
from keras.api.applications.xception import preprocess_input
from sklearn.model_selection import train_test_split
import glob

### Define path
Tentukan/definisikan folder untuk training data. Folder terbagi menjadi dua,  **Fake** dan **Real**. Gambar yang **palsu/tidak valid** dimasukkan ke dalam folder **Fake** dan di-*assign* ke variabel *fake_image_dir*, dan gambar yang **asli/valid** dimasukkan ke dalam folder **Real** yang kemudian di-*assign* ke dalam variabel *real_image_dir*.

In [ ]:
# Define paths to your data
real_image_dir = 'C:/Users/afria/Downloads/ml-backup/real'
fake_image_dir = 'C:/Users/afria/Downloads/ml-backup/fake'

### Load model
Inisiasi model, dalam kasus ini menggunakan model Xception yang sudah di-*training* menggunakan [ImageNet](https://www.image-net.org/).

In [ ]:
# Load the Xception model, pretrained on ImageNet
xception_model = Xception(weights='imagenet', include_top=False, pooling='avg')

## Pendifinisian Fungsi

### Fungsi load_and_preprocess_image

Fungsi ini berguna untuk loading dan reisizing/mengatur ukuran gambar menjadi ukuran X kali Y pixel. Dalam kasus ini, ukuran gambar diubah menjadi 299x299 pixel. Jika tidak ada gambar yang dimuat, maka program akan mengeluarkan sebuah *Error Warning*. Fungsi ini mengambil parameter path dan image_array.

In [ ]:
def load_and_preprocess_image(path=None, image_array=None):
    if path:
        img = cv2.imread(path)
        x = cv2.resize(img, (299, 299))
    elif image_array is not None:
        x = cv2.resize(image_array, (299, 299))
    else:
        raise ValueError("Either path or image_array must be provided.")
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

### Fungsi extract_features

Fungsi ini berguna untuk mengekstrak fitur-fitur atau ciri-ciri dari gambar yang sudah dipilah antara Fake dan Real menggunakan model [Xception](https://keras.io/api/applications/xception/). Fungsi ini mengambil parameter image_array.

In [ ]:
def extract_features(image_array):
    img = load_and_preprocess_image(image_array=image_array)
    features = xception_model.predict(img)
    return features

### Fungsi ocr_check
Fungsi ini berguna untuk mendeteksi teks pada gambar yang diproses dengan menggunakan [Tesseract](https://github.com/tesseract-ocr/tesseract). Pada kasus ini, ocr_check akan mengecek kehadiran teks *faceswap* dan *manycams* dalam gambar. Jika ditemukan, fungsi ini akan menghasilkan nilai **True**. Fungsi ini mengambil parameter image.

In [ ]:
def ocr_check(image):
    text = pytesseract.image_to_string(image)
    text = text.lower()  # Convert text to lowercase for easier matching
    if "faceswap" in text or "manycam" in text:
        return True
    return False

### Fungsi load_images_from_directory
Fungsi ini berguna untuk memuat atau mengambil gambar dari *directory*. Fungsi ini akan mengambil seluruh file yang ada dalam sebuah *directory* untuk kemudian diproses. Gambar yang diambil oleh fungsi ini akan dimasukkan ke dalam sebuah *Array* menjadi bentuk image_array. Fungsi ini mengambil parameter directory.

In [ ]:
def load_images_from_directory(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(('.png:Zone.Identifier', '.jpg:Zone.Identifier')):
            continue
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        if img is not None:
            features = extract_features(img)
            features = np.squeeze(features)
            images.append(features)
            label = 1 if directory == real_image_dir else 0
            labels.append(label)
    return np.array(images), np.array(labels)

### Fungsi train_model
Fungsi ini berguna untuk melatih model agar dapat membedakan antara mana gambar KYC nasabah yang valid dan yang tidak valid. *User* tidak perlu mengajarkan program apa saja ciri-ciri dari KYC nasabah yang tidak valid, karena program memiliki fungsi train_model yang bisa mengajari dirinya sendiri.

In [ ]:
def train_model():
    real_images, real_labels = load_images_from_directory(real_image_dir)
    fake_images, fake_labels = load_images_from_directory(fake_image_dir)
    images = np.concatenate((real_images, fake_images))
    labels = np.concatenate((real_labels, fake_labels))
    if images.size == 0 or labels.size == 0:
        print("No images loaded. Please check your directories.")
        return
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
    model = Sequential([
        Input(shape=(2048,)),  # Adjust this to 2048 to match the Xception output
        Dense(512, activation='relu'),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=16)
    model.save('image_classifier_model.keras')
    print("Model training complete and saved.")

### Fungsi update_model
Fungsi ini berguna untuk memperbarui model setelah program menerima user input bila ada sebuah prediksi yang salah. Hal ini diimplementasikan guna mempercepat proses pembelajaran program. Fungsi ini mengambil parameter features dan correct_label.

In [ ]:
def update_model(features, correct_label):
    try:
        model = load_model('image_classifier_model.keras')
        X_train = np.expand_dims(features, axis=0)
        y_train = np.array([correct_label])
        model.fit(X_train, y_train, epochs=1, batch_size=1, verbose=1)
        model.save('image_classifier_model.keras')
        print("Model updated and saved with the corrected label: ", correct_label)
    except Exception as e:
        print("Error loading model: ", e)

### Fungsi process_image_test
Fungsi ini berguna untuk mengetes sebuah gambar secara individu. Dalam fungsi ini, program dapat meminta validasi dari *user* untuk memastikan apakah prediksi program benar atau salah. Fungsi ini mengambil parameter model dan img_path.

In [ ]:
def process_image_test(model, img_path):
    if not os.path.exists(img_path):
        print("Error: The file does not exist.")
        return
    image = cv2.imread(img_path)
    if image is None:
        print("Error: Unable to load the image.")
        return

    # Perform OCR check before extracting features
    if ocr_check(image):
        print("Warning: Detected text indicating 'faceswap' or 'manycam'. This image may be manipulated.")
        return

        # Extract features and make a prediction
    features = extract_features(image)
    features = np.squeeze(features)  # Remove extra dimension
    score = model.predict(np.expand_dims(features, axis=0))[0][0]
    print(f"Prediction confidence score: {score:.4f}")

    # Output the result
    if score >= 0.5:
        print("Gambar ini diklasifikasikan sebagai asli.")
        predicted_label = 1
    else:
        print("Gambar ini diklasifikasikan sebagai palsu.")
        predicted_label = 0

    # Ask for user confirmation
    correct = input("Apakah prediksi benar? (y/t): ").strip().lower()
    if correct == 't':
        correct_label = 0 if predicted_label == 1 else 1
        update_model(features, correct_label)
    else:
        print("No update needed.")

### Fungsi test_model
Fungsi ini berguna untuk memunculkan prompt yang akan ditunjukkan kepada *user* sebelum *user* memilih file gambar yang akan kemudian dites secara individu.

In [ ]:
def test_model():
    """Test the model using an image provided by the user."""
    # Load the trained model
    try:
        model = load_model('image_classifier_model.keras')
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Get the image path from the user
    img_path = input("Enter the image path to test: ").strip()

    if not os.path.exists(img_path):
        print(f"Error: File {img_path} tidak ditemukan.")
        return

    image = cv2.imread(img_path)
    if image is None:
        print(f"Error: Tidak dapat memuat gambar. Silakan cek path file dan keberadaan file tersebut.")
        return

    process_image_test(model, img_path)

### Fungsi test_multiple_images
Fungsi ini berguna untuk mengetes banyak gambar sekaligus. Namun, yang membedakan fungsi ini dengan fungsi process_image_test yang hanya mengetes satu gambar, adalah fungsi ini tidak menerima feedback user atas prediksi yang dibuat program. Hal ini dilakukan agar proses penyesuaian prediksi yang berasal dari input user tidak terjadi dalam jumlah banyak secara sekaligus yang berujung menjadi tidak efisien dibandingkan dengan memberikan input kepada satu gambar saja.

In [ ]:
def test_multiple_images():
    try:
        model = load_model('image_classifier_model.keras')
        file_paths = choose_multiple_images()  # Using updated CLI function
        if not file_paths:
            print("No images provided.")
            return

        for img_path in file_paths:
            if not os.path.exists(img_path):
                print(f"Error: File {img_path} not found.")
                continue
            image = cv2.imread(img_path)
            if image is None:
                print("Error: Unable to load image.")
                continue
            features = extract_features(image)
            features = np.squeeze(features)
            score = model.predict(np.expand_dims(features, axis=0))[0][0]
            print(f"Image: {img_path} - Kemungkinan gambar valid: {score * 100:.2f}%")
            if score >= 0.5:
                print("Gambar ini valid.")
            else:
                print("Gambar ini tidak valid.")
    except Exception as e:
        print("Error loading model:", e)

### Fungsi choose_multiple_images
Fungsi ini berguna untuk memunculkan prompt yang akan ditunjukkan kepada *user* sebelum *user* memilih banyak file gambar yang akan kemudian dites secara bersamaan.

In [ ]:
def choose_multiple_images():
    print("Please enter the paths of the images you want to test, separated by commas.")
    print("You can also enter a directory path to select all images in that folder.")

    img_paths = input("Enter image paths or a directory path: ").strip().split(',')
    print(img_paths)
    selected_files = []

    for path in img_paths:
        path = path.strip()

        if os.path.isdir(path):
            # If path is a directory, select all images in that directory
            image_files = glob.glob(os.path.join(path, "*.[pjPJ][pnPN][gG]"))
            selected_files.extend(image_files)
        elif os.path.isfile(path):
            # If path is a file, just add it to the list
            selected_files.append(path)

    if not selected_files:
        print("No valid image paths provided.")
    else:
        print(f"Selected {len(selected_files)} images.")

    return selected_files


### Fungsi main_menu
Fungsi berguna untuk menampilkan menu utama secara terus-menerus bahkan setelah user selesai memilih pilihan untuk melatih model atau test model. Fungsi ini diimplementasikan untuk mempermudah *User Experience* dalam menggunakan program ini.

In [ ]:
def main_menu():
    while True:
        print("\nChoose an option:")
        print("1. Train the model")
        print("2. Test a single image")
        print("3. Test multiple images")
        print("4. Exit")
        choice = input("Enter your choice (1-4): ")
        if choice == '1':
            train_model()
        elif choice == '2':
            test_model()
        elif choice == '3':
            test_multiple_images()
        elif choice == '4':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please enter a valid option (1-4).")

## Kode utama
Berikut adalah kode utama yang akan dieksekusi secara terus-menerus hingga *user* memilih opsi 4 untuk keluar dari menu.

In [ ]:
if __name__ == "__main__":
    main_menu()